In [1]:
import pandas as pd

In [2]:
def dist_condicionada(data,col,cond,rev=False,dropna=True):
    
    """ Distribución de la columna col condicionada por la columna cond y diferencias relativas con la
    distribución no condicionada de la variable col.
    Si rev=True, distribución de cond condicionada por col.
    Si dropna=True, prescinde de registros NaN."""
    
    df=data[[col,cond]]

    # Permutación de col y cond con rev==True:
    if rev==True:
        temp=col
        col=cond
        cond=temp
    
    # Para incluir NaNs en la cuenta con groupby: 
    if dropna==False: 
        df=df.astype(str)
        
    # Frecuencias relativas condicionadas:
    fr=pd.DataFrame(df.groupby(cond)[col].value_counts(normalize=True))
    fr.rename(columns={fr.columns[-1]:'1. fr. relativas condicionadas'},inplace=True)
    
    # Columna con distribución no condicionada de variable col (frecuencias relativas):
    no_cond=pd.DataFrame(df[col].value_counts(normalize=True))
    no_cond.rename(columns={no_cond.columns[-1]:'no condicionada'},inplace=True)
    fr=fr.merge(no_cond,left_on=col,right_index=True)
    
    # Columna de diferencias entre distribuciones condicionadas y no condicionada:
    fr['dif']=fr['1. fr. relativas condicionadas'].sub(fr['no condicionada'])
    
    # Columna auxiliar de diferencias relativas (respecto a frecuencias no condicionadas):
    fr['3. dif relativas']=fr['dif'].div(fr['no condicionada'])
    
    pt=pd.pivot_table(data=fr,index=col,columns=[cond],\
                      values=['1. fr. relativas condicionadas','3. dif relativas'])
    pt.sort_index(inplace=True)
    
    # Nueva columna de frecuencias no condicionadas (más sencillo que ajustar la auxiliar):
    pt['2. fr relativas '+str(col)]=df[col].value_counts(normalize=True)
    
    # Reordenación de columnas:
    pt=pt[pt.sort_index(axis=1,ascending=True).columns]
    
    # Formato: 
    n=df[cond].nunique()
    m=pt.shape[1]
    d_style=dict(zip(pt.columns,['{:,.2}']*(m-n)+['{:+,.2%}']*n))
    pt=pt.style.format(d_style)
    
    return pt

In [3]:
# Ilustración: 

df=pd.DataFrame({'Nombre': ['Lola',None,'Lola','Lola','Juana','Juana','Erika','Filomena','Filomena','Filomena'],
        'Edad':['edad 1','edad 2','edad 2','edad 3','edad 2','edad 2','edad 1','edad 2','edad 3',None]})
df

,Nombre,Edad
0,Lola,edad 1
1,None,edad 2
2,Lola,edad 2
3,Lola,edad 3
4,Juana,edad 2
5,Juana,edad 2
6,Erika,edad 1
7,Filomena,edad 2
8,Filomena,edad 3
9,Filomena,None


In [4]:
dist_condicionada(df,'Nombre','Edad',dropna=False)

In [5]:
dist_condicionada(df,'Nombre','Edad')

In [6]:
dist_condicionada(df,'Nombre','Edad',rev=True)